In [40]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.9"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.9"

//using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.9 Microsoft.Extensions.Configuration.Json, 9.0.9

In [41]:
// Import the Semantic Kernel NuGet packages 
#r "nuget: Microsoft.Extensions.AI"
#r "nuget: Microsoft.SemanticKernel, 1.65.0" 
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.65.0"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.65.0-preview"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.65.0"

using System.Net.Http;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Increase the default timeout for the HttpClient to 200 seconds
var aiHttpClient = new HttpClient();
aiHttpClient.Timeout = TimeSpan.FromSeconds(200);

var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        serviceId: "azureOpenAI",
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        httpClient: aiHttpClient,
        apiKey: azureOpenAIAPIKey!
        )
    .Build();

Installed Packages Microsoft.Extensions.AI, 9.9.0 Microsoft.SemanticKernel, 1.65.0 Microsoft.SemanticKernel.Agents.Abstractions, 1.65.0 Microsoft.SemanticKernel.Agents.Core, 1.65.0 Microsoft.SemanticKernel.Agents.OpenAI, 1.65.0-preview

In [ ]:
using Azure.AI.OpenAI;
using Microsoft.Extensions.AI;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.ClientModel;

// Retrieve the Azure OpenAI Endpoint from Semantic Kernel:
var skChatCompletionService = semanticKernel.GetRequiredService<IChatCompletionService>("azureOpenAI");
IChatClient chatClient = skChatCompletionService.AsChatClient();

var azureOpenAIClient = new AzureOpenAIClient(new Uri(azureOpenAIEndpoint!), new ApiKeyCredential(azureOpenAIAPIKey!));

#pragma warning disable SKEXP0130
#pragma warning disable OPENAI001

// Retrieve the Azure OpenAI Responses Client from the Azure OpenAI Client
var azureOpenAIResponsesClient = azureOpenAIClient.GetOpenAIResponseClient(azureOpenAIModelDeploymentName!);


var agentOptions = new OpenAIResponseAgentInvokeOptions()
{
    ResponseCreationOptions = new()
    {
        MaxOutputTokenCount = 10000,
        ReasoningOptions = new()
        {
            ReasoningEffortLevel = OpenAI.Responses.ResponseReasoningEffortLevel.Medium,
            // Concise is not available in Azure OpenAI 
            ReasoningSummaryVerbosity = OpenAI.Responses.ResponseReasoningSummaryVerbosity.Detailed,
        }
    }
};

In [39]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.  
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Register the agent with the Semantic Kernel. 
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
OpenAIResponseAgent dwightEisenhowerAgent =
    new(azureOpenAIResponsesClient)
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions
    };

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPrompt = """
Who are you and? What are some things you think you can help with? How should I consider using you?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionMessage = new ChatMessageContent(AuthorRole.User, decisionPrompt);

// STEP 4: Invoke the Dwight Eisenhower Decision Agent
var dwightEisenhowerAgentReasoning = $"""
### Agent Reasoning:

""";
var dwightEisenhowerAgentResponse = $"""

### Agent Response:

""";

#pragma warning disable SKEXP0110
Microsoft.SemanticKernel.Agents.AgentThread? thread = null;
await foreach (var item in dwightEisenhowerAgent.InvokeAsync(chatDecisionMessage, thread, agentOptions))
{
    foreach (var content in item.Message.Items)
    {
        // This currently doesn't work as thinking and non-thinking content is mixed as `TextContent` only.
        if (content is ReasoningContent thinking)
        {
            dwightEisenhowerAgentReasoning += thinking.Text;
        }
        else if (content is Microsoft.SemanticKernel.TextContent text)
        {
            dwightEisenhowerAgentResponse += text.Text;
        }
    }
}
// Display the Agent's reasoning and response
dwightEisenhowerAgentReasoning.DisplayAs("text/markdown");
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

### Agent Reasoning:



### Agent Response:
| Who I am | Things I can help with (examples) | How to consider using me | Best practices for working with me | Follow-up questions I’ll ask to help you best |
|---|---|---|---|---|
| I am a Decision Intelligence strategist AI: a structured, systems-thinking assistant that helps you break down problems, generate options, weigh trade-offs, and create practical action plans. I lean on Eisenhower-style prioritization and clear decision frameworks. | - Strategic prioritization (what to do now vs later)<br>- Problem framing and root-cause analysis<br>- Generating and comparing options (pros, cons, risks)<br>- Designing decision frameworks and criteria (including Eisenhower matrix)<br>- Scenario planning and contingency mapping<br>- Risk assessment and mitigation plans<br>- Project scoping, timelines, and implementation checklists<br>- Communication/rollout messaging and stakeholder alignment<br>- Data synthesis into decision-ready summaries | Use me as a structured thinking partner: provide context, goals, constraints, stakeholders, and deadlines. Ask me to: frame the decision, list options, quantify trade-offs, build prioritization matrices, draft plans or scripts, or design experiments to test assumptions. Iterate: start with a high-level brief, then refine with data and feedback. | - Give clear goal(s), timeframe, constraints, and key stakeholders.<br>- Share data or assumptions when available; highlight uncertainty.<br>- Tell me your decision criteria (e.g., cost, speed, quality, risk tolerance).<br>- Ask for specific outputs (matrix, checklist, draft email, plan).<br>- Use my outputs as input to human judgment—validate critical facts and legal/medical/financial items with experts.<br>- Prefer shorter cycles: decide, act, measure, adjust. | 1) What decision or problem are you trying to solve (one sentence)?<br>2) What outcome(s) would count as success and by when?<br>3) What constraints (budget, team, timeline, regulations) matter most?<br>4) Who are the primary stakeholders and their priorities?<br>5) What information or data do you already have, and where are the gaps? |